> `hover` filters can stack together.
>
> :speedboat: This makes selections incredibly powerful.

-   <details open><summary>This page addresses **single components** of `hover`</summary>
    We are using code snippets to pick out parts of the annotation interface, so that the documentation can explain what they do.

    -   Please be aware that this is NOT how one would typically use `hover`.
    -   Typical usage deals with [recipes](../../tutorial/t1-active-learning) where the individual parts have been tied together.

</details>

-   <details open><summary>Dependencies for {== local environments ==}</summary>
    When you run the code locally, you may need to install additional packages.

    To run the text embedding code on this page, you need:
```shell
    pip install spacy
    python -m spacy download en_core_web_md
```

    To render `bokeh` plots in Jupyter, you need:
```shell
    pip install jupyter_bokeh
```

    If you are using JupyterLab older than 3.0, use this instead ([reference](https://pypi.org/project/jupyter-bokeh/)):
```shell
    jupyter labextension install @jupyter-widgets/jupyterlab-manager
    jupyter labextension install @bokeh/jupyter_bokeh
```

</details>

## **Preparation**

As always, start with a ready-for-plot dataset:

In [1]:
from hover.core.dataset import SupervisableTextDataset
import pandas as pd

raw_csv_path = "https://raw.githubusercontent.com/phurwicz/hover-gallery/main/0.5.0/20_newsgroups_raw.csv"
train_csv_path = "https://raw.githubusercontent.com/phurwicz/hover-gallery/main/0.5.0/20_newsgroups_train.csv"

# for fast, low-memory demonstration purpose, sample the data
df_raw = pd.read_csv(raw_csv_path).sample(400)
df_raw["SUBSET"] = "raw"
df_train = pd.read_csv(train_csv_path).sample(400)
df_train["SUBSET"] = "train"
df_dev = pd.read_csv(train_csv_path).sample(100)
df_dev["SUBSET"] = "dev"
df_test = pd.read_csv(train_csv_path).sample(100)
df_test["SUBSET"] = "test"

# build overall dataframe and ensure feature type
df = pd.concat([df_raw, df_train, df_dev, df_test])
df["text"] = df["text"].astype(str)

# this class stores the dataset throught the labeling process
dataset = SupervisableTextDataset.from_pandas(df, feature_key="text", label_key="label")

🔵 SupervisableTextDataset: Initializing...

🔵 SupervisableTextDataset: Deduplicating...

🔵 SupervisableTextDataset: --subset raw rows: 400 -> 369.

🔵 SupervisableTextDataset: --subset train rows: 400 -> 382.

🔵 SupervisableTextDataset: --subset dev rows: 100 -> 95.

🔵 SupervisableTextDataset: --subset test rows: 100 -> 96.

🟢 SupervisableTextDataset: Set up label encoder/decoder with 20 classes.

🟢 SupervisableTextDataset: Population updater: latest population with 20 classes.

🟢 SupervisableTextDataset: SupervisableTextDataset: finished initialization.

<br>

In [2]:
import spacy
import re
from functools import lru_cache

# use your preferred embedding for the task
nlp = spacy.load("en_core_web_md")

# raw data (str in this case) -> np.array
@lru_cache(maxsize=int(1e+4))
def vectorizer(text):
    clean_text = re.sub(r"[\s]+", r" ", str(text))
    return nlp(clean_text, disable=nlp.pipe_names).vector

# any kwargs will be passed onto the corresponding reduction
# for umap: https://umap-learn.readthedocs.io/en/latest/parameters.html
# for ivis: https://bering-ivis.readthedocs.io/en/latest/api.html
reducer = dataset.compute_nd_embedding(vectorizer, "umap", dimension=2)

Vectorizing: 100%|██████████| 942/942 [00:02<00:00, 349.12it/s]


🔵 SupervisableTextDataset: Fit-transforming UMAP on 846 samples...

🔵 SupervisableTextDataset: Transforming UMAP on 96 samples...

🟢 SupervisableTextDataset: Computed 2-d embedding in columns ['embed_2d_0', 'embed_2d_1']

<br>

## **Soft-Label Explorer**

Active learning works by predicting labels and scores (i.e. soft labels) and utilizing that prediction. An intuitive way to plot soft labels is to color-code labels and use opacity ("alpha" by `bokeh` terminology) to represent scores.

`SoftLabelExplorer` delivers this functionality:

In [3]:
from bokeh.io import show, output_notebook

output_notebook()

# normally your would skip notebook_url or use Jupyter address
notebook_url = 'localhost:8888'

from hover.recipes.subroutine import standard_softlabel
from bokeh.layouts import row, column

softlabel = standard_softlabel(dataset)
show(softlabel.figure, notebook_url=notebook_url)

Loading BokehJS ...

🔵 BokehTextSoftLabel: Setting up DataFrames

🔵 BokehTextSoftLabel: Setting up sources

🔵 BokehTextSoftLabel: Setting up widgets

🔵 BokehTextSoftLabel: activated search base response on TextInput(id='p1114', ...)

...

🟢 BokehTextSoftLabel: Plotted subset dev with 95 points

🟢 BokehTextSoftLabel: Plotted subset raw with 369 points

🟢 BokehTextSoftLabel: Plotted subset train with 382 points

🟢 SupervisableTextDataset: Subscribed BokehTextSoftLabel to dataset pushes: {'raw': 'raw', 'train': 'train', 
'dev': 'dev'}

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



<br>

## **Filter Selection by Score Range**

Similarly to `finder`, a `softlabel` plot has its own selection filter. The difference lies in the filter condition:

In [4]:
show(softlabel.score_filter, notebook_url=notebook_url)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



<br>

## **Linked Selections & Joint Filters**

When we plot multiple `explorer`s for the same `dataset`, it makes sense to synchronize selections between those plots. `hover` recipes take care of this synchronization.

-   :tada: This also works with cumulative selections. Consequently, the cumulative toggle is synchronized too.

Since each filter is narrowing down the selections we make, joint filters is just set intersection, extended

-   from two sets (original selection + filter)
-   to N sets (original selection + filter A + filter B + ...)

The [`active_learning` recipe]((../t1-active-learning/)) is built of `softlabel + annotator + finder`, plus a few widgets for iterating the model-in-loop.

In the next tutorial(s), we will see more recipes taking advantage of linked selections and joint filters. Powerful indeed!